In [1]:
import pandas as pd
import numpy as np
import re

from bokeh.plotting import figure, show
from bokeh.tile_providers import get_provider, STAMEN_TONER, CARTODBPOSITRON_RETINA
from bokeh.models import HoverTool, FreehandDrawTool, BoxEditTool, ColumnDataSource

### For some reason bokeh maps uses Web Mercator coordinates
* Define function to switch from lat/long to mercator x/y coordinates

In [2]:
# Define function to switch from lat/long to mercator x/y coordinates
def to_mercator(lat, lon):
    
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

#### Ames, Iowa latitude-longitude (42.034534, -93.620369), converted to mercator

In [3]:
# Ames, Iowa lat long, converted to mercator
Ames_center = to_mercator(42.034534, -93.620369)

In [80]:
geo = pd.read_csv('geodata.csv')
geo.shape

(2624, 2)

#### Convert all house lat_long to mercator coordinates

In [81]:
#list of house mercator coordinates
x_merc = []
y_merc = []

for each in geo['lat_long']:
    ll = re.sub("[^0-9.\- ]","",each).split()
    #if 0, append center of Ames (-10421771.804958373, 5166153.874296733)
    if ll == ['0']:
        x_merc.append(Ames_center[0])
        y_merc.append(Ames_center[1])
    else:
        x, y = to_mercator(float(ll[0]),float(ll[1]))
        x_merc.append(x)
        y_merc.append(y)

In [82]:
geo = pd.concat([geo, pd.Series(x_merc), pd.Series(y_merc)],axis=1).set_index('PID')

In [83]:
geo = geo.rename(columns={0:'x_merc',1:'y_merc'})

#### Create list of landmarks in Ames

In [84]:
landmarks = {'landmarks':['Iowa State University',
                          'Municipal Airport',
                          'North Grand Mall',
                          'Mary Greeley Medical Center',
                          'Jack Trice Stadium'],
            'x_merc':[to_mercator(42.0267,-93.6465)[0],
                      to_mercator(41.9987,-93.6223)[0],
                      to_mercator(42.0494,-93.6224)[0],
                      to_mercator(42.0323,-93.6111)[0],
                      to_mercator(42.0140,-93.6359)[0]],
            'y_merc':[to_mercator(42.0267,-93.6465)[1],
                      to_mercator(41.9987,-93.6223)[1],
                      to_mercator(42.0494,-93.6224)[1],
                      to_mercator(42.0323,-93.6111)[1],
                      to_mercator(42.0140,-93.6359)[1]]}

marks = pd.DataFrame(landmarks)

### Draw Map of Houses and Landmarks

In [9]:
background = get_provider(CARTODBPOSITRON_RETINA) #CARTODBPOSITRON_RETINA, STAMEN_TONER
x_zoom = 7000
y_zoom = 5000

# Base Map Layer
fig = figure(plot_width=1200, plot_height=800,
             x_range=(Ames_center[0]-x_zoom, Ames_center[0]+y_zoom), 
             y_range=(Ames_center[1]-x_zoom, Ames_center[1]+y_zoom),
             x_axis_type="mercator", y_axis_type="mercator",
             title="Ames Iowa Housing Map")
fig.add_tile(background)

# Dots for Houses
fig.circle(x="x_merc", y="y_merc",
         size=4,
         fill_color="orange", line_color='dodgerblue',
         fill_alpha=0.6,
         source=geo)

# Big Dots for Landmarks, with Hover interactivity
my_hover = HoverTool(names=['landmark'])
my_hover.tooltips = [('X', '@landmarks')]
fig.circle(x="x_merc", y="y_merc",
           size=10,
           fill_color="tomato", line_color='tomato',
           fill_alpha=0.3,
           name='landmark',
           source=marks)
fig.add_tools(my_hover)

# # Boxes
# sectordata = ColumnDataSource(
#     data={'x': [], 'y': [], 'width': [], 'height': []})

# mybox = fig.rect('x', 'y', 'width', 'height', color='pink', source=sectordata, alpha=0.4)
# boxes = BoxEditTool(renderers=[mybox],num_objects=5)
# fig.add_tools(boxes)

# Add draw tool
renderer = fig.multi_line([[1, 9]], [[5, 5]], line_width=4, alpha=0.4, color='red')
draw_tool = FreehandDrawTool(renderers=[renderer], num_objects=4)
fig.add_tools(draw_tool)

show(fig)

![citymap](Ames.png)

### Since data has neighborhood information, recode neighborhoods to bigger city "sectors"
* as per  https://www.thinkames.com/maps/

In [20]:
data = pd.read_csv('ALL_data.csv', index_col='PID')
data.Neighborhood.value_counts()

NAmes      417
CollgCr    240
OldTown    217
Edwards    169
Somerst    145
Gilbert    145
Sawyer     141
NWAmes     124
NridgHt    123
SawyerW    114
Mitchel    105
BrkSide    104
Crawfor     94
IDOTRR      80
NoRidge     67
Timber      54
StoneBr     45
SWISU       42
ClearCr     40
MeadowV     38
BrDale      30
Blmngtn     23
NPkVill     23
Veenker     23
Blueste     10
Greens       8
GrnHill      2
Landmrk      1
Name: Neighborhood, dtype: int64

In [21]:
data = data.Neighborhood.to_frame().copy()

#### Create recoding dictionary

In [98]:
code = {'Blmngtn': 'NO', 'Blueste':'SW', 'BrDale': 'NO',
        'BrkSide':'DT', 'ClearCr': 'SW', 'CollgCr':'SW',
        'Crawfor':'SW', 'Edwards':'SW', 'Gilbert': 'NO',
        'IDOTRR':'DT', 'MeadowV':'SE', 'Mitchel':'SE',
        'NAmes':'NO', 'NoRidge':'NW', 'NPkVill':'NO',
        'NridgHt':'NW', 'NWAmes':'NO', 'OldTown':'DT',
        'SWISU':'SW', 'Sawyer':'NW', 'SawyerW':'NW',
        'Somerst':'NW', 'StoneBr':'NO', 'Timber':'SW',
        'Veenker':'NW', 'Greens':'NW', 'GrnHill':'SW', 'Landmrk':'DT'}

# NW            NorthWest
# NO            North
# NE            NorthEast (No Houses)
# SW            SoutWest
# DT            Downtown
# SO            South     (No Houses)
# SE            SouthEast

* Create new Column with House Sector Data

In [99]:
data['Sector'] = [code[each] for each in data.Neighborhood]

In [100]:
data.Sector.value_counts()

NO    807
SW    651
NW    621
DT    402
SE    143
Name: Sector, dtype: int64

In [101]:
assert data.Sector.value_counts().agg('sum') == data.Neighborhood.count()

In [102]:
# Set Sector Colors for Map
color_dict = {'NO':'green',
              'SW':'purple',
              'NW':'yellow',
              'DT':'dodgerblue',
              'SE':'red',
              'SO':'black'}
data['Color'] = [color_dict[each] for each in data.Sector]

#### Join to geo DF

In [103]:
map_data = geo.merge(data,how='left',on='PID')

### Map again with Sector Colors

In [105]:
background = get_provider(CARTODBPOSITRON_RETINA) #CARTODBPOSITRON_RETINA, STAMEN_TONER
x_zoom = 7000
y_zoom = 5000

# Base Map Layer
fig = figure(plot_width=1200, plot_height=800,
             x_range=(Ames_center[0]-x_zoom, Ames_center[0]+y_zoom), 
             y_range=(Ames_center[1]-x_zoom, Ames_center[1]+y_zoom),
             x_axis_type="mercator", y_axis_type="mercator",
             title="Ames Iowa Housing Map")
fig.add_tile(background)

# Dots for Houses
my_hover = HoverTool(names=['House'])
my_hover.tooltips = [('PID', '@Neighborhood')]
fig.circle(x="x_merc", y="y_merc",
           size=5,
           color='Color', line_color='black', line_width = 0.5,
           fill_alpha=0.6,
           name='House',
           source=map_data)
fig.add_tools(my_hover)

# Big Dots for Landmarks, with Hover interactivity
my_hover = HoverTool(names=['landmark'])
my_hover.tooltips = [('X', '@landmarks')]
fig.circle(x="x_merc", y="y_merc",
           size=10,
           fill_color="tomato", line_color='tomato',
           fill_alpha=0.3,
           name='landmark',
           source=marks)
fig.add_tools(my_hover)


# Add draw tool
renderer = fig.multi_line([[1, 9]], [[5, 5]], line_width=4, alpha=0.4, color='red')
draw_tool = FreehandDrawTool(renderers=[renderer], num_objects=4)
fig.add_tools(draw_tool)

show(fig)